In [4]:
import sys
!{sys.executable} -m pip install torchvision

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------- ----------------------------- 0.3/1.2 MB 6.8 MB/s eta 0:00:01
     ---------------------------- ----------- 0.8/1.2 MB 8.8 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 8.4 MB/s eta 0:00:00
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/91/6e/db0e545302bf93b6dbbdc496dd192c7f8e8c3bb1584acba069256d8b51d4/charset_normalizer-3.2.0-cp311-cp311-win_amd64.whl.metadata
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.met

In [5]:
import torch
import torchvision.models as models

In [6]:
base_model = models.mobilenet_v2(pretrained=True)

base_model = base_model.features

c:\Users\aadit\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aadit\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\aadit/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class LandmarkHead(nn.Module):
    def __init__(self):
        super(LandmarkHead, self).__init__()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(1280, 128)  # 1280 is the number of channels from MobileNetV2 features
        self.fc2 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

landmark_head = LandmarkHead()


In [8]:
class FacialLandmarkModel(nn.Module):
    def __init__(self, base_model, landmark_head):
        super(FacialLandmarkModel, self).__init__()
        self.base_model = base_model
        self.landmark_head = landmark_head

    def forward(self, x):
        x = self.base_model(x)
        x = self.landmark_head(x)
        return x

model = FacialLandmarkModel(base_model, landmark_head)


In [10]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)